# IMDB Movie Lookup System
## Author: Ahria Dominguez
### Last Updated: 6/29/2024

#### Import libraries.

In [1]:
# Imports the necessary libraries.
import urllib.request, urllib.parse, urllib.error
import json
import os

#### Obtain a key and store it in JSON as *APIkeys.json*. 
To keep my API key from being shown, I already used json.dump to create a .json file with my API key in a dictionary. You can dump your API key into the same file name to use the rest of the code.

#### Load the secret API key from a JSON file stored in the same folder in a variable, by using *json.loads*. Open the _APIkeys.json_ file.

In [2]:
# 'json.loads' does not work here since we're loading in a file and not a string, so I will use 'json.load'.
# Opens the same file name as above in a read-only format and assigns it to 'api_key' for future access. It
# also assigns just the API key number to a variable to use later.
with open('APIkeys.json', 'r') as json_file:
    api_file = json.load(json_file)
    my_key = api_file['api_key']
    
# To protect my key, I will not be printing off the results to show it loaded. However, I did check to make
# sure it worked, then deleted the cells.

#### Assign the OMDb portal as a string to a variable.

In [3]:
# Assigns the portal URL to 'serviceurl' since it says to reference it like that later on.
serviceurl = 'http://www.omdbapi.com/?'

#### Create a variable called *apikey* with the last portion of the URL (*&apikey=secretapikey*), where *secretapikey* is your own API key.

In [4]:
# Assigns the API key portion of the URL to 'apikey'.
apikey = '&apikey=' + my_key

#### Write a utility function called *print_json* to print the movie data from a JSON file.

In [5]:
# Creates the 'print_json' function to print the value data from the list of keys that are given when
# calling for information from the API. I found these keys by going to the API website, going to 'Examples',
# scrolling down and entering the 'Titanic' movie's information under 'By Title', and writing down the keys
# that are given in the response. I ignored the 'Poster' and 'Response' keys since I'll access them later.
def print_json(movie):
    keys = ['Title', 'Year', 'Rated', 'Released', 'Runtime', 'Genre', 'Director', 'Writer', 
               'Actors', 'Plot', 'Language', 'Country', 'Awards', 'Ratings', 
               'Metascore', 'imdbRating', 'imdbVotes', 'imdbID', 'Type', 'DVD', 'BoxOffice', 'Production',
              'Website']
    
    print("_" * 100) # Prints a line to break up the information.
    # Loops through the indices in the above list and prints the keys and values if they are there. If a key
    # does not exist for a movie, it will print "Not a key for this movie" in the value spot. This is good for
    # error handling.
    for k in keys:
        value = movie.get(k, "Not a key for this movie.")
        print(f"{k}: {movie[k]}")
    print("_" * 100) # Prints more lines to break up the space.

#### Write a utility function to download a poster of the movie based on the information from the JSON dataset and save it in your local folder. Use the *os* module. The poster data is stored in the JSON key *Poster*. Use the Python command to open a file and write the poster data. Close the file after you're done. This function will save the poster data as an image file.

In [6]:
# Creates the 'download_poster' function to access the poster URL, download the image of the poster, and save
# it to a new folder in the current directory. 
def download_poster(movie):
    # Accesses the movie's Title key.
    title = movie['Title']
    # Accesses the movie's poster URL key.
    poster_url = movie['Poster']
    # Prints the poster URL for error checking.
    print(f"Poster URL: {poster_url}")
    
    # Finds the ending of the poster URL to know how to format the file (e.g., '.jpg', '.png', etc.)
    file_extension = poster_url.split('.')[-1]
    # Reads the images using the URL.
    poster_data = urllib.request.urlopen(poster_url).read()
    
    # Assigns the path to save the file to be in the current directory but under a new file (or already
    # created one if it's already been run) called 'Posters'.    
    save_file = os.path.join(os.getcwd(), 'Posters')
    # Tells Python to make the new file directory if it isn't already there.
    if not os.path.isdir(save_file):
        os.mkdir(save_file)
    # Assigns the name of the file using the file path, title, and file extension to 'file_name'.
    file_name = os.path.join(save_file, f"{title}.{file_extension}")
    
    # Uses a try-except block to open and write the file name variable, writes the poster data into that file,
    # indicates whether it worked or not with a printed message (and an error if it did not work), prints
    # where the file was saved (the path), and then closes the file. 
    try:
        with open(file_name, 'wb') as fhand:
            fhand.write(poster_data)
        print(f"Poster was downloaded to the directory: {file_name}")
    except Exception as error:
        print(f"There was an error saving the poster: {error}")
        
# It's better to use "with open" rather than open(file_name) and file_name.close() because
# it will do it for you once the loop is done. Then, you won't forget to close it. 

#### Write a utility function called *search_movie* to search for a movie by its name, print the downloaded *JSON* data, and save the movie poster in the local folder. Use a *try-except* loop for this. Use the previously created *serviceurl* and *apikey* variables. You have to pass on a dictionary with a key, *t*, and the movie name as the corresponding value to the *urllib.parse.urlencode()* function and then add the *serviceurl* and *apikey* to the output of the function to construct the full URL. This URL will be used to access the data. The JSON data has a key called *Response*. If it is *True*, that means the read was successful. Check this before processing the data. If it's not successful, then print the *JSON* key *Error*, which will contain the appropriate error message returned by the movie database.

In [7]:
# Creates a function called 'search_movie' that searches for the desired movie, checks if the movie read was 
# successful (using 'Response' == True/False), prints the dictionary values of the movie information (see 
# 'print_json' function), downloads the movie poster (see 'download_poster' function), and prints any errors 
# that were encountered.
def search_movie(title):
    # Begins a try-except block to set the title of the movie as a parameter to the URL, as well as the API
    # key, sets the URL, and prints a statement that the title is being accessed. When testing the code, you
    # could also have it print out the URL being accessed, but it shouldn't stay in the code if other people
    # will access it, since it will have your API key. 
    try:
        parameter = {'t': title}
        url = f"{serviceurl}{urllib.parse.urlencode(parameter)}{apikey}"
        print(f'Information on "{title}:"')
        # Opens the URL thorugh urllib.request and reads the data.
        uh = urllib.request.urlopen(url)
        data = uh.read()
        # Loads the JSON data into the 'movie_data' variable.
        movie_data = json.loads(data)
        
        # Checks if the 'Response' key's value is True and then uses the 'print_json' function on the movie
        # data if it is.
        if movie_data['Response'] == 'True':
            print_json(movie_data)
            # Also checks to see if the 'Poster' key is not empty before performing the 'download_poster' 
            # function on the data.
            if movie_data['Poster'] != 'N/A':
                download_poster(movie_data)
            else:
                pass
        else:
            print("There was an error accessing the movie data: ", movie_data['Error'])
    
    # Ends the try-except block by handling any errors that may have been encountered through accessing the 
    # URL and prints the error reason.
    except urllib.error.URLError as error:
        print(f"Sorry! There was an error: {error.reason}")

#### Test the *search_movie* function by entering *Titanic*.

In [8]:
# Searches for the movie "Titanic" using the function created above.
search_movie("Titanic")

Information on "Titanic:"
____________________________________________________________________________________________________
Title: Titanic
Year: 1997
Rated: PG-13
Released: 19 Dec 1997
Runtime: 194 min
Genre: Drama, Romance
Director: James Cameron
Writer: James Cameron
Actors: Leonardo DiCaprio, Kate Winslet, Billy Zane
Plot: A seventeen-year-old aristocrat falls in love with a kind but poor artist aboard the luxurious, ill-fated R.M.S. Titanic.
Language: English, Swedish, Italian, French
Country: United States, Mexico
Awards: Won 11 Oscars. 126 wins & 83 nominations total
Ratings: [{'Source': 'Internet Movie Database', 'Value': '7.9/10'}, {'Source': 'Rotten Tomatoes', 'Value': '88%'}, {'Source': 'Metacritic', 'Value': '75/100'}]
Metascore: 75
imdbRating: 7.9
imdbVotes: 1,287,662
imdbID: tt0120338
Type: movie
DVD: N/A
BoxOffice: $674,292,608
Production: N/A
Website: N/A
____________________________________________________________________________________________________
Poster URL: h

#### Test the *search_movie* function by entering *"Random_error"*.

In [9]:
# Searches for a non-existent movie "Random_error" to show that the function can handle bad responses.
search_movie("Random_error")

Information on "Random_error:"
There was an error accessing the movie data:  Movie not found!
